In [1]:
import warnings
warnings.filterwarnings("ignore")
import os
import numpy as np
from collections import defaultdict
import pickle
import tensorflow as tf
import matplotlib.pyplot as plt

In [ ]:
vgg_range = dict()
vgg_range['input_shape'] = [128,1024,15,0]
vgg_range['input_size'] = [10,19,10,1] # Logspace 9
vgg_range['vgg_layers'] = [2,10,9,0]
vgg_range['vgg_layers_size'] = [1,7,7,0] 
vgg_range['filters'] = [4,10,7,1] # Logspace 8
vgg_range['hidden_layers_size'] = [1,10,10,0]
vgg_range['hidden_layers'] = [2,3.5,4,2] # Logspace 7
vgg_range['output_shape'] = [1,10,10,1] # Logspace 10
vgg_range['batch_size'] = [3,10,8,1] #Logspace 8

vgg_params = dict()
for key in vgg_range.keys():
    val = vgg_range[key]
    if val[-1] == 0:
        vgg_params[key] = np.linspace(val[0],val[1],num=val[2]).astype('int')
    elif val[-1] == 1:
        vgg_params[key] = np.logspace(val[0],val[1],num=val[2],base=2).astype('int')
    elif val[-1] == 2:
        vgg_params[key] = np.logspace(val[0],val[1],num=val[2],base=10).astype('int')

inception_range = dict()
inception_range['input_shape'] = [128,1024,15,0]
inception_range['input_size'] = [10,19,10,1] # Logspace 9
inception_range['inception_layers'] = [1,5,5,0]
inception_range['f1'] = [64,320,5,0]
inception_range['f2_in'] = [128,384,5,0]
inception_range['f2_out'] = [192,448,5,0]
inception_range['f3_in'] = [32,160,5,0]
inception_range['f3_out'] = [32,160,5,0]
inception_range['f4_out'] = [32,160,5,0]
inception_range['hidden_layers_size'] = [1,10,10,0]
inception_range['hidden_layers'] = [2,3.5,4,2] # Logspace 7
inception_range['output_shape'] = [1,10,10,1] # Logspace 10
inception_range['batch_size'] = [3,10,8,1] #Logspace 8

inception_params = dict()
for key in inception_range.keys():
    val = inception_range[key]
    if val[-1] == 0:
        inception_params[key] = np.linspace(val[0],val[1],num = val[2]).astype('int')
    elif val[-1] == 1:
        inception_params[key] = np.logspace(val[0],val[1],num = val[2],base = 2).astype('int')
    elif val[-1] == 2:
        inception_params[key] = np.logspace(val[0],val[1],num = val[2],base = 10).astype('int')
        
resnet_range = dict()
resnet_range['input_shape'] = [128,1024,15,0]
resnet_range['input_size'] = [10,19,10,1] # Logspace 9
resnet_range['resnet_layers'] = [3,7,5,0]
resnet_range['hidden_layers_size'] = [1,10,10,0]
resnet_range['hidden_layers'] = [2,3.5,4,2] # Logspace 7
resnet_range['output_shape'] = [1,10,10,1] # Logspace 10
resnet_range['batch_size'] = [3,10,8,1] #Logspace 8

resnet_params = dict()
for key in resnet_range.keys():
    val = resnet_range[key]
    if val[-1] == 0:
        resnet_params[key] = np.linspace(val[0],val[1],num = val[2]).astype('int')
    elif val[-1] == 1:
        resnet_params[key] = np.logspace(val[0],val[1],num = val[2],base = 2).astype('int')
    elif val[-1] == 2:
        resnet_params[key] = np.logspace(val[0],val[1],num = val[2],base = 10).astype('int')


fc_range = dict()
fc_range['input_shape'] = [128,1024,15,0]
fc_range['input_size'] = [10,19,10,1] # Logspace 9
fc_range['hidden_layers'] = [1,10,10,0]
fc_range['output_shape'] = [1,10,10,1] # Logspace 10
fc_range['batch_size'] = [3,10,8,1] #Logspace 8

fc_params = dict()
for key in fc_range.keys():
    val = fc_range[key]
    if val[-1] == 0:
        fc_params[key] = np.linspace(val[0],val[1],num = val[2]).astype('int')
    elif val[-1] == 1:
        fc_params[key] = np.logspace(val[0],val[1],num = val[2],base = 2).astype('int')
    elif val[-1] == 2:
        fc_params[key] = np.logspace(val[0],val[1],num = val[2],base = 10).astype('int')
        
params = dict()
params['vgg'] = vgg_params
params['resnet'] = resnet_params
params['inception'] = inception_params
params['fc'] = fc_params

def get_time_train_data(folder_name):
    x = []
    y = []
    model_params = params[folder_name]
    for key in model_params:
        for value in model_params[key]:
            filename = f'{folder_name}/{folder_name}-{key}-{value}.pickle'
            if os.path.exists(filename):
                with open(filename, 'rb') as handle:
                    output_config = pickle.load(handle)
                    x.append(list(output_config['flops_param'].values()) + 
                             list(output_config['layers_param'].values()) + 
                             list(output_config['weights_param'].values()))
                    x[-1].append(output_config['input_size'])
                    x[-1].append(output_config['batch_size'])
                    y.append(list(output_config['train_times']))
    x = np.asarray(x).astype('float64')
    y = np.asarray(y).astype('float64')
    y = np.mean(y,axis=1)
    return x,y

def get_single_time_train_data(output_config):
    x = []
    y = []
    x.append(list(output_config['flops_param'].values()) + 
             list(output_config['layers_param'].values()) + 
             list(output_config['weights_param'].values()))
    x[-1].append(output_config['input_size'])
    x[-1].append(output_config['batch_size'])
    y.append(list(output_config['train_times']))
    x = np.asarray(x).astype('float64')
    y = np.asarray(y).astype('float64')
    y = np.mean(y,axis=1)
    return x,y

def get_trained_model(folder_name):
    x,y = get_time_train_data(folder_name)
    min_error = float('inf')
    min_model = None
    for i in range(100):
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25)
        rf = RandomForestRegressor(n_estimators=20)
        rf.fit(x_train, y_train)
        y_pred_train = rf.predict(x_train)
        y_pred_test = rf.predict(x_test)
        train_error = mean_absolute_percentage_error(y_train,y_pred_train)
        test_error = mean_absolute_percentage_error(y_test,y_pred_test)
        if test_error < min_error:
            min_loss = test_error
            min_model = rf
        if (test_error - train_error) < 0.05 and max(test_error,train_error) < 0.1:
            break
    y_pred_train = min_model.predict(x_train)
    y_pred_test = min_model.predict(x_test)
    print(f'{folder_name} - train MSE : {mean_absolute_percentage_error(y_train,y_pred_train)}')
    print(f'{folder_name} - test MSE : {mean_absolute_percentage_error(y_test,y_pred_test)}')
    
    return min_model

models = dict()
models['vgg'] = get_trained_model('vgg')
models['inception'] = get_trained_model('inception')
models['resnet'] = get_trained_model('resnet')
models['fc'] = get_trained_model('fc')

In [ ]:
file_to_read = open('vgg/vgg-filters-128.pickle', "rb")
data = pickle.load(file_to_read)
x,y = get_single_time_train_data(data)
y_pred = models['vgg'].predict(x)
print(y_pred)

In [ ]:
def get_model_stored_train_times(dir_path):
    # Necessary Imports
    data_dict = defaultdict(list)

    for file_name in os.listdir(dir_path):
        file_path = os.path.join(dir_path, file_name)
        file_name_elems = file_name.split('-')
        varying_attr_name = file_name_elems[1]
        varying_attr_val = int(file_name_elems[2][:file_name_elems[2].find('.')])

        # Forming a list of tuples for each of the varying attributes...
        if varying_attr_name not in data_dict.keys():
            data_dict[varying_attr_name] = []

        if os.path.isfile(file_path):
            file_to_read = open(file_path, "rb")
            data = pickle.load(file_to_read)
            x,y = get_single_time_train_data(data)
            y_pred = models[dir_path].predict(x)[0]
            data_dict[varying_attr_name].append([varying_attr_val, np.mean(data['train_times']), y_pred])

    # Sorting the list of tuples to get cleaner analysis
    for varying_attr_name, varying_attr_vals in data_dict.items():
        data_dict[varying_attr_name] = sorted(varying_attr_vals, key = lambda x : x[0])

    return data_dict

In [ ]:
# VGG
dir_path = os.path.join("vgg")
vgg_data_dict = get_model_stored_train_times(dir_path)
print(vgg_data_dict)

In [ ]:
# ResNet
dir_path = os.path.join("resnet")
resnet_data_dict = get_model_stored_train_times(dir_path)
print(resnet_data_dict)

In [ ]:
# Inception
dir_path = os.path.join("inception")
inception_data_dict = get_model_stored_train_times(dir_path)
print(inception_data_dict)

In [ ]:
# fc
dir_path = os.path.join("fc")
fc_data_dict = get_model_stored_train_times(dir_path)
print(fc_data_dict)

In [ ]:
# Visualize all the data for the training times
def plot_varying_attrs_vs_train_times(model_name, dict_data):
    import matplotlib.pyplot as plt
    import seaborn as sns

#     plt.rcParams["figure.figsize"] = (15,40)
    num_plots = len(dict_data)
    fig, axs = plt.subplots(num_plots,figsize=(15,num_plots*5))
    print(f'Training Times (y-axis) for Models (with varying attributes): {model_name.upper()}')

    plt_counter = 0
    for varying_attr_name, varying_attr_vals in dict_data.items():
        x_vals = [attr[0] for attr in varying_attr_vals]
        y_vals = [attr[1] for attr in varying_attr_vals]
        y_pred_vals = [attr[2] for attr in varying_attr_vals]
        axs[plt_counter].set_title(varying_attr_name)
        axs[plt_counter].plot(x_vals,y_vals,c='blue',label='Actual')
        axs[plt_counter].plot(x_vals,y_pred_vals,c='orange',label='Predicted')
        axs[plt_counter].legend()
#         sns.barplot(x_vals, y_vals, ax = axs[plt_counter])
        plt_counter += 1
    
    # Saving the image
    plt.savefig(os.path.join("Visualizations", f'{model_name}_data_vs.png'))

In [ ]:
# VGG Visualizations
plot_varying_attrs_vs_train_times('VGG', vgg_data_dict)

In [ ]:
# ResNet Visualizations
plot_varying_attrs_vs_train_times('ResNet', resnet_data_dict)

In [ ]:
# Inception Visualizations
plot_varying_attrs_vs_train_times('Inception', inception_data_dict)

In [ ]:
# fc Visualizations
plot_varying_attrs_vs_train_times('Fully Connected', fc_data_dict)

In [ ]:
def get_model_train_times_per_aux_vars(dir_path):
    # Necessary Imports
    import os
    import numpy as np
    from collections import defaultdict
    import pickle

    data_dict = defaultdict(dict)

    for file_name in os.listdir(dir_path):
        if file_name.find("input_shape") == -1 and file_name.find("layers") == -1:
            continue
        file_path = os.path.join(dir_path, file_name)
        file_name_elems = file_name.split('-')
        varying_attr_name = file_name_elems[1]

        # Forming a dict of list of tuples for each of the varying attributes...
        if varying_attr_name not in data_dict.keys():
            data_dict[varying_attr_name] = dict()

        if os.path.isfile(file_path):
            file_to_read = open(file_path, "rb")
            data = pickle.load(file_to_read)
            
            x,y = get_single_time_train_data(data)
            y_pred = models[dir_path].predict(x)[0]

            total_flops = sum(list(data['flops_param'].values()))
            if 'flops_param' not in data_dict[varying_attr_name].keys():
                data_dict[varying_attr_name]['flops_param'] = []
            data_dict[varying_attr_name]['flops_param'].append((total_flops, np.mean(data['train_times']),y_pred))

            trainable_params = data['weights_param']['trainable']
            if 'trainable' not in data_dict[varying_attr_name].keys():
                data_dict[varying_attr_name]['trainable'] = []
            data_dict[varying_attr_name]['trainable'].append((trainable_params, np.mean(data['train_times']),y_pred))

            total_depth = sum(list(data['layers_param'].values()))
            if 'layers_param' not in data_dict[varying_attr_name].keys():
                data_dict[varying_attr_name]['layers_param'] = []
            data_dict[varying_attr_name]['layers_param'].append((total_depth, np.mean(data['train_times']),y_pred))

    # Sorting the list of tuples to get cleaner analysis
    for varying_attr_name, varying_attr_vals in data_dict.items():
        for varying_attr_val_name, varying_data in varying_attr_vals.items():
            data_dict[varying_attr_name][varying_attr_val_name] = sorted(varying_data, key = lambda x : x[0])

    return data_dict

In [ ]:
# VGG
dir_path = os.path.join("vgg")
vgg_data_dict_2 = get_model_train_times_per_aux_vars(dir_path)
print(vgg_data_dict_2)

In [ ]:
# ResNet
dir_path = os.path.join("resnet")
resnet_data_dict_2 = get_model_train_times_per_aux_vars(dir_path)
print(resnet_data_dict_2)

In [ ]:
# Inception
dir_path = os.path.join("inception")
inception_data_dict_2 = get_model_train_times_per_aux_vars(dir_path)
print(inception_data_dict_2)

In [ ]:
# fc
dir_path = os.path.join("fc")
fc_data_dict_2 = get_model_train_times_per_aux_vars(dir_path)
print(fc_data_dict_2)

In [ ]:
aux_vars = ['flops_param', 'trainable', 'layers_param']

In [ ]:
# Visualize all the data for the training times
def plot_varying_aux_attrs_vs_train_times(model_name, dict_data):
    import matplotlib.pyplot as plt
    import seaborn as sns

#     plt.rcParams["figure.figsize"] = (15,80)
    num_plots = len(dict_data)*len(aux_vars)
    fig, axs = plt.subplots(num_plots,figsize=(15,num_plots*5))
    print(f'Training Times (y-axis) for Models (with varying attributes): {model_name.upper()}')


    plt_counter = 0
    for varying_attr_name, varying_attr_vals in dict_data.items():
        for varying_attr_val_name, varying_attr_data in varying_attr_vals.items():
            x_vals = [x[0] for x in varying_attr_data]
            y_vals = [x[1] for x in varying_attr_data]
            y_pred_vals = [x[2] for x in varying_attr_data]
            axs[plt_counter].set_title(f"{varying_attr_val_name} w.r.t {varying_attr_name}")
            axs[plt_counter].plot(x_vals,y_vals,c='blue',label='Actual')
            axs[plt_counter].plot(x_vals,y_pred_vals,c='orange',label='Predicted')
            axs[plt_counter].legend()
#             sns.barplot(x_vals, y_vals, ax = axs[plt_counter])
            plt_counter += 1
    
    # Saving the image
    plt.savefig(os.path.join("Visualizations", f'{model_name}_aux_vars_data_vs.png'))

In [ ]:
# VGG Aux Variable Visualizations
plot_varying_aux_attrs_vs_train_times('VGG', vgg_data_dict_2)

In [ ]:
# ResNet Aux Variable Visualizations
plot_varying_aux_attrs_vs_train_times('ResNet', resnet_data_dict_2)

In [ ]:
# Inception Aux Variable Visualizations
plot_varying_aux_attrs_vs_train_times('Inception', inception_data_dict_2)

In [ ]:
# fc Aux Variable Visualizations
plot_varying_aux_attrs_vs_train_times('Fully Connected', fc_data_dict_2)